In [1]:
import torch
import clip
import pandas as pd
import numpy as np
import json

In [2]:
available_models = clip.available_models()
print(available_models)

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [3]:
print('\nLoading model...')

clip_model = 'ViT-B/16'

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clip_model, device=device, jit=False)
print(f"Done! Model {clip_model} loaded to {device} device")


Loading model...
Done! Model ViT-B/16 loaded to cuda device


In [4]:
img_embs = pd.read_pickle('../data/fface_val_img_embs.pkl')

txt_embs = torch.load('../data/synms-gender-labels.pt')

with open('../data/synms_gender_labels.json', encoding='utf-8') as json_data:
    data = json.load(json_data)
    fface_classes = list(data.keys())
    prompts = list(data.values())
    man_prompts = prompts[0]
    woman_prompts = prompts[1]
    fface_prompts = man_prompts + woman_prompts

fface_df = pd.read_csv('../data/fface_val.csv')

In [5]:
def get_similarities(img, txts):
    """Grab similarity between text and image embeddings."""
    image_features = torch.from_numpy(img).to('cuda')
    similarity = 100.0 * image_features @ txts.T

    return similarity

In [28]:
def get_sims_dict(img_embs, txt_prompts):
    final_dict = {}
    for _, emb in img_embs.iterrows():
        name = emb['file']
        img_features = emb['embeddings']
        img_sims = get_similarities(img_features, txt_embs)
        sims_dict = {}
        for label, score in zip(txt_prompts, img_sims[0]):
            sims_dict[label] = score.cpu().numpy().item()
        final_dict[name] = sims_dict
    return final_dict

In [47]:
def get_top_synm(final_dict):
    files = final_dict.keys()
    wins = []
    for val in final_dict.values():
        scores_list = list(val.values())
        label_list = list(val.keys())
        np_scores = np.asarray(scores_list)
        windex = np.where(np_scores == np_scores.max())[0][0]
        wins.append(label_list[windex])

    top_synm_dict = {'file': files, 'winner': wins}
    return top_synm_dict

In [13]:
def get_sum_synms(final_dict):
    files = final_dict.keys()
    preds = []
    
    for key, val in final_dict.items():
        man_score = 0
        woman_score = 0
        for k, v in val.items():
            if k in man_prompts:
                man_score += v
            else:
                woman_score += v
        preds.append(f'Male' if man_score > woman_score else f'Female')

    sum_dict = {'file': files, 'preds': preds}
    return(sum_dict)

In [49]:
def synm_to_gender(synm):
    if synm in man_prompts:
        return 'Male'
    else:
        return 'Female'

In [29]:
sims_dict = get_sims_dict(img_embs=img_embs, txt_prompts=fface_prompts)

In [55]:
sum_dict = get_sum_synms(final_dict=sims_dict)
sum_df = pd.DataFrame(data=sum_dict)
print(sum_df.head())

        file   preds
0  val/1.jpg    Male
1  val/2.jpg  Female
2  val/3.jpg    Male
3  val/4.jpg  Female
4  val/5.jpg    Male


In [54]:
top_dict = get_top_synm(final_dict=sims_dict)
top_df = pd.DataFrame(data=top_dict)
print(top_df.head())

        file           winner
0  val/1.jpg              boy
1  val/2.jpg      grandmother
2  val/3.jpg  middle-aged man
3  val/4.jpg      young woman
4  val/5.jpg              man


In [53]:
bin_df = top_df.copy()
bin_df['winner'] = top_df['winner'].map(synm_to_gender)
print(bin_df.head())

        file  winner
0  val/1.jpg    Male
1  val/2.jpg  Female
2  val/3.jpg    Male
3  val/4.jpg  Female
4  val/5.jpg    Male
